In [1]:
import tensorflow as tf
import numpy as np

2022-02-17 16:20:22.801703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-17 16:20:22.801739: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
x=np.loadtxt('geom.dat')
y=np.loadtxt('TBE.dat')[:,np.newaxis]
idx=np.random.permutation(len(x))
ntrain=1000
nvalidate=1000
x_train=x[idx[:ntrain]]
y_train=y[idx[:ntrain]]
x_validate=x[idx[ntrain:ntrain+nvalidate]]
y_validate=y[idx[ntrain:ntrain+nvalidate]]
batch_size=4
normalization=tf.keras.layers.Normalization(axis=None)
normalization.adapt(x)
training_set=tf.data.Dataset.from_tensor_slices((normalization(x_train), y_train)).shuffle(256).batch(batch_size)
validation_set=tf.data.Dataset.from_tensor_slices((normalization(x_validate), y_validate)).shuffle(256).batch(batch_size)

2022-02-17 16:20:25.509108: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2022-02-17 16:20:25.509164: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Lab): /proc/driver/nvidia/version does not exist
2022-02-17 16:20:25.509355: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(y.shape[-1], activation='linear')
])
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,loss='mse')
csv_logger=tf.keras.callbacks.CSVLogger('training.log')
history=model.fit(training_set, epochs=1024, verbose=1, validation_data=validation_set, callbacks=csv_logger)

np.savetxt('result',np.concatenate((x,y,model(x).numpy()),axis=1))

Epoch 1/1024
250/250 [==============================] - 1s 2ms/step - loss: 177765200.0000 - val_loss: 74428384.0000
Epoch 2/1024
250/250 [==============================] - 0s 1ms/step - loss: 72178904.0000 - val_loss: 75125552.0000
Epoch 3/1024
250/250 [==============================] - 0s 1ms/step - loss: 71998184.0000 - val_loss: 74284192.0000
Epoch 4/1024
250/250 [==============================] - 0s 1ms/step - loss: 71743648.0000 - val_loss: 74330352.0000
Epoch 5/1024
250/250 [==============================] - 0s 1ms/step - loss: 71621968.0000 - val_loss: 75367888.0000
Epoch 6/1024
250/250 [==============================] - 0s 1ms/step - loss: 71937288.0000 - val_loss: 73970416.0000
Epoch 7/1024
250/250 [==============================] - 0s 1ms/step - loss: 71615280.0000 - val_loss: 73906896.0000
Epoch 8/1024
250/250 [==============================] - 0s 1ms/step - loss: 71791976.0000 - val_loss: 74387400.0000
Epoch 9/1024
250/250 [==============================] - 0s 1ms/step - l